# Making masks

In [2]:
from imports import os, np, nib, pd, ThreadPoolExecutor, resize, imageio, time, torch

In [ ]:
# make array of 15k entries with the region id of each, to relate to the region where each entry in the array
# after applying the first mask is

ROI = {'rest': [1, 419],
    '17networks_LH_VisualA_ExStr': [420, 450],           #region 1
           '17networks_LH_VisualA_PrC': [451, 452],
           '17networks_LH_VisualA_SPL': [453, 454],
           '17networks_LH_VisualB_ExStrInf': [455, 463],
           '17networks_LH_VisualB_ExStrSup': [464, 477],
'17networks_LH_VisualB_Striate': [478, 488],                #region 2
'17networks_LH_VisualC_ExStr': [489, 496],                  #region 3
'17networks_LH_VisualC_Striate': [497, 500],                #region 4
'rest2': [501, 909],
'17networks_RH_VisualA_ExStr': [910, 940],                  #region 5
           '17networks_RH_VisualA_PrC': [941, 942],
           '17networks_RH_VisualA_SPL': [943, 949],
           '17networks_RH_VisualB_ExStrInf': [950, 959],
           '17networks_RH_VisualB_ExStrSup': [960, 971],
           '17networks_RH_VisualB_SPL': [972, 975],
'17networks_RH_VisualB_Striate': [976, 991],                #region 6
'17networks_RH_VisualC_ExStr': [992, 1000]}                 #region 7

ROI2 = {'a': [0, 1]}

# load complete schaefer mask
schaefer = 'Schaefer2018_1000Parcels_Kong2022_17Networks_order_FSLMNI152_2mm.nii.gz'
img_schaefer = nib.load(schaefer)
data_schaefer = np.asanyarray(img_schaefer.dataobj)

region_id = 0
big_mask = np.zeros((91, 109, 91))
region_labels = np.zeros(902629, dtype=int)
#filling voxels with the value of their region
for area in ROI.keys():
    region_id += 1
    mask = np.zeros((91, 109, 91))
    mask += ((data_schaefer >= ROI[area][0]) & (data_schaefer <= ROI[area][1])).astype(int)
    big_mask += ((data_schaefer >= ROI[area][0]) & (data_schaefer <= ROI[area][1])).astype(int)
    mask_2d = mask.reshape(-1,)
    indices = np.where(mask_2d == 1)[0]
    region_labels[indices] = region_id

big_mask_2d = big_mask.reshape(-1,)
indices = np.where(big_mask_2d == 1)[0]
#region_labels = region_labels[indices]
#np.save('region_ids_15364.npy', region_labels)
np.save('region_ids_all2.npy', region_labels)

In [11]:
# remove the same indices of the region_ids as the indices removed by the snr filter from the 
# 15k entries array for voxels

mask_snr = np.load('mask_schaefer1000_4609.npy', mmap_mode = 'r')
region_ids = np.load('region_ids_900k.npy')
print(mask_snr.shape)
print(region_ids.shape)

#mask_3d = np.load('mask_schaefer1000_4609.npy', mmap_mode = 'r')
mask_2d = mask_snr.reshape(-1,)
# identify the relevant voxels to keep 
indices = np.where(mask_2d == 1)[0]
print(indices.shape)
region_ids = region_ids[indices]
print(region_ids.shape)
np.save('region_ids_4609.npy', region_ids)

non_zero_count = (mask_snr != 0).sum()
print(f"Number of non-zero values: {non_zero_count}")

(91, 109, 91)
(902629,)
(4609,)
(4609,)
Number of non-zero values: 4609


In [22]:
region_ids = np.load('region_ids_4609.npy')
print(region_ids)
#non_zero_count = (region_ids != 0).sum()
#print(f"Number of non-zero values: {non_zero_count}")

[5 5 5 ... 1 1 1]


In [29]:
def turn_off_regions(input_fmri, regions):
    #input_fmri should be an array with a shape [N, 4609], where N is number of TRs
    #regions should be like [1,4,6,7] if we want to turn off regions 1, 4, 6 and 7, according to:
#    '17networks_LH_VisualA_ExStr': [420, 450],                  #region 1
#    '17networks_LH_VisualB_Striate': [478, 488],                #region 2
#    '17networks_LH_VisualC_ExStr': [489, 496],                  #region 3
#    '17networks_LH_VisualC_Striate': [497, 500],                #region 4
#    '17networks_RH_VisualA_ExStr': [910, 940],                  #region 5
#    '17networks_RH_VisualB_Striate': [976, 991],                #region 6
#    '17networks_RH_VisualC_ExStr': [992, 1000]}                 #region 7

    region_ids = np.load('region_ids_4609.npy')
    #region_ids is like [5,5,5,4,1,1,1,1,2,2,...] meaning that the first 3 voxels of the input_fmri arrays
    #                                             are from region 5, the following one from region 4, 
    #                                             following four from region 1, etc

    indices = []
    for region_id in regions:
#        indices += np.where(region_ids == region_id)[0]
        indices.extend(np.where(region_ids == region_id)[0].tolist())
    
    #input_fmri[indices] = 0
    fmri2 = input_fmri.copy()
    fmri2[:, indices] = 0
    return fmri2





In [ ]:
from dataset import *
from models_new_2 import *
from visualisation_new_2 import *

dataset_ID = 6661 # ID of a specific dataset. 6661 refer to preprocessed data with a mask of shape (4609,). 6660 refers to preprocessed data with a mask of shape (15364,)
mask_size = 4609 # number of voxels in the preprocessed fMRI data. either 4609 or 15364
trainset, valset, testset = get_dataset(dataset_ID, mask_size) # data are loaded into dictionaries

print(trainset['fMRIs'].shape)
non_zero_count = (trainset['fMRIs'] != 0).sum()
print("non zero count =", non_zero_count)
regions = [1,2,3,4,5,6,7]
fmri_regions_off = turn_off_regions(trainset['fMRIs'], regions)
print(fmri_regions_off.shape)
non_zero_count = (fmri_regions_off != 0).sum()
print("non zero count after off =", non_zero_count)

#alright seems to be working !


(4321, 4609)
non zero count = 19915489
(4321, 4609)
non zero count after off = 0


In [ ]:
def test_new_decoder(real=True, model_name='decoder_4609_1650', test_on_train=False, test_input = testset['fMRIs'], test_label = testset['videos']):
    '''
    Tests the decoder
    if real=False (default) -> tests on brain activity coming from encoder
    if real=True -> tests on real brain activity
    '''
    print("testing decoder", model_name)

    #load decoder part of encoder decoder
    model = Decoder(mask_size)
    #save_model_as = 'decoder_4609_50'
    #save_model_as = 'decoder_4609_1650'
    state_dict = torch.load(model_name)
    model.load_state_dict(state_dict)
    #model = model1.decoder

    #load data
#    if real:
#        test_input = testset['fMRIs']
#        test_label = testset['videos']
#    else:
    if not real:
        test_input = model1.encoder(testset['videos'])
        test_label = testset['videos']

    if test_on_train:
        num_samples = trainset["fMRIs"].shape[0]

        # Generate random indices based on that number
        random_indices = np.random.choice(num_samples, size=30, replace=False)

        # Create the testset with random samples
        testset2 = {
            "fMRIs": {
                "test": trainset["fMRIs"][random_indices]  # Shape will be (30, 4609)
            },
            "videos": {
                "test": trainset["videos"][random_indices]  # Shape will be (30, 3, 112, 112, 32)
            }
        }
        test_input = testset2['fMRIs']
        test_label = testset2['videos']


    criterion = D_Loss()
    device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
    pretrained_decoder=None
    model_to_test='decoder'
    statistical_testing = False
    display_plots = True
    save_plots = True

    test_model(test_input, test_label, model, criterion, device, pretrained_decoder, model_to_test, statistical_testing, display_plots, save_plots, model_name=model_name)
    return

def test_decoder_with_regions_off(real=True, model_name='decoder_4609_1650', test_on_train=False, test_input = testset['fMRIs'], test_label = testset['videos'], regions):
    
    fmri_regions_off = test_input.copy()
    for video_name in test_input.keys:
        fmri_regions_off[video_name] = turn_off_regions(test_input[video_name], regions)

    test_new_decoder(real, model_name, test_on_train, fmri_regions_off, test_label = testset['videos'])



### nevermind the code below

In [3]:
data_path = 'fMRIs_schaefer1000_4609'
done = []
for subject in os.listdir(data_path):
    if subject[:5] == 'sub-S' and len(subject) == 7 and int(subject[5:7]) not in done:
        subject_path = os.path.join(data_path, subject)
        files = os.listdir(subject_path)
        npy_files = []

        for f in files:
            if f.endswith('.npy'):
                npy_files.append(f)

        for npy_file in npy_files:
            file_path = os.path.join(subject_path, npy_file)
        
            # Load only the header information
            array_header = np.lib.format.open_memmap(file_path, mode='r')
            shape = array_header.shape
        
            # Print information
            print(f"File: {file_path}")
            print(f"Shape: {shape}")
            print("-" * 50)

        

File: fMRIs_schaefer1000_4609/sub-S04/Sintel.npy
Shape: (4609, 712)
--------------------------------------------------
File: fMRIs_schaefer1000_4609/sub-S04/Payload.npy
Shape: (4609, 937)
--------------------------------------------------
File: fMRIs_schaefer1000_4609/sub-S04/TearsOfSteel.npy
Shape: (4609, 608)
--------------------------------------------------
File: fMRIs_schaefer1000_4609/sub-S04/Superhero.npy
Shape: (4609, 947)
--------------------------------------------------
File: fMRIs_schaefer1000_4609/sub-S04/BigBuckBunny.npy
Shape: (4609, 531)
--------------------------------------------------
File: fMRIs_schaefer1000_4609/sub-S04/FirstBite.npy
Shape: (4609, 617)
--------------------------------------------------
File: fMRIs_schaefer1000_4609/sub-S04/BetweenViewings.npy
Shape: (4609, 776)
--------------------------------------------------
File: fMRIs_schaefer1000_4609/sub-S04/AfterTheRain.npy
Shape: (4609, 536)
--------------------------------------------------
File: fMRIs_sc

KeyboardInterrupt: 

In [ ]:
ROI = {'17networks_LH_VisualA_ExStr': [420, 450],
           #'17networks_LH_VisualA_PrC': [451, 452],
           #'17networks_LH_VisualA_SPL': [453, 454],
           #'17networks_LH_VisualB_ExStrInf': [455, 463],
           #'17networks_LH_VisualB_ExStrSup': [464, 477],
'17networks_LH_VisualB_Striate': [478, 488],
'17networks_LH_VisualC_ExStr': [489, 496],
'17networks_LH_VisualC_Striate': [497, 500],
'17networks_RH_VisualA_ExStr': [910, 940],
           #'17networks_RH_VisualA_PrC': [941, 942],
           #'17networks_RH_VisualA_SPL': [943, 949],
           #'17networks_RH_VisualB_ExStrInf': [950, 959],
           #'17networks_RH_VisualB_ExStrSup': [960, 971],
           #'17networks_RH_VisualB_SPL': [972, 975],
'17networks_RH_VisualB_Striate': [976, 991],
'17networks_RH_VisualC_ExStr': [992, 1000]}

def print_areas(ROI):
    print(ROI.keys())

def turn_off_areas(ROI, areas_names):
    ROI2 = ROI.copy()

    for area in areas_names:
        if area in ROI2:
            del ROI2[area]
    
    # load complete schaefer mask
    schaefer = 'Schaefer2018_1000Parcels_Kong2022_17Networks_order_FSLMNI152_2mm.nii.gz'
    img_schaefer = nib.load(schaefer)
    data_schaefer = np.asanyarray(img_schaefer.dataobj)

    mask = np.zeros((91, 109, 91))
    for area in ROI2.keys():
        mask += ((data_schaefer >= ROI2[area][0]) & (data_schaefer <= ROI2[area][1])).astype(int)
    
    mask_2d = mask.reshape(-1,)
    # identify the relevant voxels to keep 
    indices = np.where(mask_2d == 1)[0]
    print(f'Using a mask covering {indices.shape[0]} voxels.')

    # create the folder to store the data if it doesn't exist
    main_folder = 'temp_masked_fmri'
    os.makedirs('temp_masked_fmri', exist_ok=True)

    data_path = 'fMRIs_schaefer1000_4609'
    done = []
    for subject in os.listdir(data_path):
        if subject[:5] == 'sub-S' and len(subject) == 7 and int(subject[5:7]) not in done:
            print(f'Preprocessing subject {subject}.')
            subject_path = os.path.join(data_path, subject)
            folder = os.path.join(main_folder, subject)
            os.makedirs(folder, exist_ok=True)

            MNI_paths = list(Path(subject_path).rglob('filtered_func_data_res_MNI.nii'))
            for MNI_path in MNI_paths:
                name = str(MNI_path)[91:-36]     # movie name
                img = nib.load(MNI_path)
                data_4d = np.asanyarray(img.dataobj)

                time_points = data_4d.shape[3]
                data_2d = data_4d.reshape(-1, time_points)     # flatten the fMRI data
                masked_fMRI = data_2d[indices].astype(np.float32)     # extract the relevant voxels only
                #data_2d shape:    (902629, 300)
                #indices shape:    (15000,)
                #masked_fMRI shape: (15000, 300)

                # save the file
                file_path = os.path.join(folder, f'{name}.npy')
                np.save(file_path, masked_fMRI)
                del img, data_4d, data_2d, masked_fMRI

    store_average_fMRIs_new(main_folder)

    print("Storing fMRIs completed. Total time: {:.2f} minutes.".format((time.time() - tic) / 60))
    print('---')
    

    
#    return ROI


def get_schaefer1000_mask(save_as):
    # load complete schaefer mask
    schaefer = 'Schaefer2018_1000Parcels_Kong2022_17Networks_order_FSLMNI152_2mm.nii.gz'
    img_schaefer = nib.load(schaefer)
    data_schaefer = np.asanyarray(img_schaefer.dataobj)

    # select 15364 voxels from the visual areas
    ROI = {'17networks_LH_VisualA_ExStr': [420, 450],
           #'17networks_LH_VisualA_PrC': [451, 452],
           #'17networks_LH_VisualA_SPL': [453, 454],
           #'17networks_LH_VisualB_ExStrInf': [455, 463],
           #'17networks_LH_VisualB_ExStrSup': [464, 477],
           '17networks_LH_VisualB_Striate': [478, 488],
           '17networks_LH_VisualC_ExStr': [489, 496],
           '17networks_LH_VisualC_Striate': [497, 500],
           '17networks_RH_VisualA_ExStr': [910, 940],
           #'17networks_RH_VisualA_PrC': [941, 942],
           #'17networks_RH_VisualA_SPL': [943, 949],
           #'17networks_RH_VisualB_ExStrInf': [950, 959],
           #'17networks_RH_VisualB_ExStrSup': [960, 971],
           #'17networks_RH_VisualB_SPL': [972, 975],
           '17networks_RH_VisualB_Striate': [976, 991],
           '17networks_RH_VisualC_ExStr': [992, 1000]}
    #basically the schaefer mask serves to identify to which "parcel"=cluster of functionally connected neurons a voxel belongs
    #and it divides the brain into 1000 parcels/regions/clusters . but we dont need to separate zones so much, so we 
    #join parcels in some bigger regions, that's why for instance '17networks_LH_VisualA_ExStr' includes voxels with 31
    #different values, it joins 31 parcels into one bigger region
    #in the schaefer mask, each voxel is assigned the unique value that corresponds to its parcel/the functional connectome it belongs to

    mask = np.zeros((91, 109, 91))  #dimensions of data_schaefer , so dimensions are 91 voxels in two directions, 109 in the remaining one
    for roi_name in ROI.keys():
        mask += ((data_schaefer >= ROI[roi_name][0]) & (data_schaefer <= ROI[roi_name][1])).astype(int)
        #for each bigger region we defined, "turn on" (=put 1, or True (it's the same)) the voxels of that region in the mask

    np.save(save_as, mask)
    print(f"Mask created (shape: {mask.shape})")